# Potenzielle Solarenergie in der Schweiz: Analyse und Vergleich

Dieses Notebook analysiert das Solarenergiepotenzial der Schweiz auf Basis von Dach- und Fassadenflächen und vergleicht es mit dem aktuellen Stromverbrauch sowie der Stromproduktion.

## Datenimport und Aufbereitung
In diesem Abschnitt werden die Rohdaten geladen, bereinigt und für die weitere Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd

# Dateipfad zur JSON-Datei mit Solarenergiepotenzialen
file_path_data = '../data/potentielle_solarenergie/Solarenergiepotenziale_Gemeinden_Daecher_und_Fassaden.json'

# Daten einlesen und in DataFrame umwandeln
data = pd.read_json(file_path_data)
df = pd.DataFrame(data)

# Nur Schweizer Gemeinden auswählen
df = df[df['Country'] == 'CH']

# Relevante Spalten auswählen und umbenennen
df = df.rename(columns={
    'Scenario1_RoofsOnly_PotentialSolarElectricity_GWh': 'RoofsOnly_GWh',
    'Scenario3_RoofsFacades_PotentialSolarElectricity_GWh': 'RoofsFacades_GWh'
})[['Canton', 'RoofsOnly_GWh', 'RoofsFacades_GWh']]

# Fassadenpotenzial berechnen (Fassaden = Dächer+Fassaden - Dächer)
df['FacadesOnly_GWh'] = df['RoofsFacades_GWh'] - df['RoofsOnly_GWh']

# Vorschau auf die ersten Zeilen
df.head()

## Visualisierung: Potenzielle Solarenergie pro Kanton

Das folgende Diagramm zeigt das Solarenergiepotenzial nach Kanton, getrennt nach Dach- und Fassadenflächen.

In [ ]:
# Plotly für die Visualisierung importieren
import plotly.graph_objects as go

# Aggregation: Potenziale pro Kanton aufsummieren
stacked_data = df.groupby("Canton")[["RoofsOnly_GWh", "FacadesOnly_GWh", "RoofsFacades_GWh"]].sum()

# Nach Gesamtpotenzial sortieren (absteigend)
stacked_data = stacked_data.sort_values(by="RoofsFacades_GWh", ascending=False)

# Farben für die Top-4-Kantone hervorheben
colors_roofs = ['rgb(0, 0, 0)' if i < 4 else 'rgb(169, 169, 169)' for i in range(len(stacked_data))]
colors_facades = ['rgb(128, 128, 128)' if i < 4 else 'rgb(211, 211, 211)' for i in range(len(stacked_data))]

# Gestapeltes Balkendiagramm erstellen
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Dächer',
    x=stacked_data.index,
    y=stacked_data["RoofsOnly_GWh"],
    marker_color=colors_roofs,
    text=(stacked_data["RoofsOnly_GWh"] / 1000).round(1).astype(str) + 'k',
    textposition='inside'
))
fig.add_trace(go.Bar(
    name='Fassaden',
    x=stacked_data.index,
    y=stacked_data["FacadesOnly_GWh"],
    marker_color=colors_facades,
    text=(stacked_data["FacadesOnly_GWh"] / 1000).round(1).astype(str) + 'k',
    textposition='inside'
))

# Layout anpassen
fig.update_layout(
    barmode='stack',
    title='Potenzielle Solarenergie in der Schweiz nach Kanton (Stand März 2025)',
    xaxis_title='Kanton',
    yaxis_title='Potenzielle Solarenergie (GWh)',
    height=500,
    template='simple_white',
    legend_title_text='Flächenkategorie'
)

# Diagramm als HTML speichern
fig.write_html("../docs/assets/diagramme/pot_solarenergie_pro_kanton.html")

# Diagramm anzeigen
fig.show()

In [ ]:
# Bibliotheken importieren
import pandas as pd
import plotly.express as px

# Strom-Endverbrauch laden und berechnen
strom_df = pd.read_csv('../data/verbrauch/Strom_Endverbrauch_Schweiz.csv', sep=';')
strom_df.columns = [col.replace(' in GWh', '') for col in strom_df.columns]
strom_endverbrauch = strom_df[strom_df['Jahr'] == strom_df['Jahr'].max()].iloc[:, 1:].sum(axis=1).values[0]

# Potenziale für Dächer und Fassaden aufsummieren
pot_solar_roofs = df['RoofsOnly_GWh'].sum()
pot_solar_facades = df['FacadesOnly_GWh'].sum()

# Vergleichs-DataFrame erstellen
vergleich_df = pd.DataFrame({
    "Kategorie": ["Dächer", "Fassaden", "Strom-Endverbrauch"],
    "Gruppe": ["Potenzielle Solarenergie", "Potenzielle Solarenergie", "Strom-Endverbrauch"],
    "GWh": [pot_solar_roofs, pot_solar_facades, strom_endverbrauch]
})

# Stromproduktion nach Energiequelle laden und berechnen
file_path_energy_data = '../data/energy_sources/elektrizitaetbilanz_jahreswerte.csv'
energy_data = pd.read_csv(file_path_energy_data, delimiter=';')
columns_to_keep = ['Jahr'] + [col for col in energy_data.columns if col.startswith('Erzeugung_')]
energy_data = energy_data[columns_to_keep]

# Wasserkraft und weitere erneuerbare Energien berechnen
energy_data['Wasserkraft'] = energy_data['Erzeugung_laufwerk_GWh'] + energy_data['Erzeugung_speicherwerk_GWh']
energy_data['Andere_erneuerbare_Energien_GWh'] = (
    energy_data['Erzeugung_holz_GWh'] +
    energy_data['Erzeugung_biogas_GWh'] +
    energy_data['Erzeugung_wind_GWh']
)

# Nur aktuellstes Jahr (2024) auswählen
energy_data = energy_data[energy_data['Jahr'] == 2024]

# Spalten umbenennen für Übersichtlichkeit
energy_data.rename(columns={
    'Erzeugung_kernkraftwerk_GWh': 'Kernkraft',
    'Erzeugung_andere_total_GWh': 'Thermische Kraft',
    'Erzeugung_photovoltaik_GWh': 'Photovoltaik',
    'Andere_erneuerbare_Energien_GWh': 'Andere erneuerbare Energien'
}, inplace=True)

# Werte extrahieren
strom_wasserkraft = energy_data['Wasserkraft'].values[0]
strom_kernkraft = energy_data['Kernkraft'].values[0]
strom_photovoltaik = energy_data['Photovoltaik'].values[0]
strom_andere_erneuerbare = energy_data['Andere erneuerbare Energien'].values[0]
strom_thermisch = energy_data['Thermische Kraft'].values[0]

# Stromquellen zur Vergleichstabelle hinzufügen
vergleich_df = pd.concat([
    vergleich_df,
    pd.DataFrame({
        "Kategorie": [
            "Wasserkraft", "Kernkraft", "Photovoltaik",
            "Andere erneuerbare Energien", "Thermische Kraft"
        ],
        "Gruppe": ["Aktuelle Stromproduktion"] * 5,
        "GWh": [
            strom_wasserkraft, strom_kernkraft, strom_photovoltaik,
            strom_andere_erneuerbare, strom_thermisch
        ]
    })
], ignore_index=True)

# Formatierte Textspalte für die Balkenbeschriftung
def make_bar_text(row):
    value = f"{row['GWh']:,.0f}".replace(",", "'") + " GWh"
    if row['Kategorie'] == "Strom-Endverbrauch":
        return value
    else:
        return f"{row['Kategorie']}<br>{value}"

vergleich_df["Text"] = vergleich_df.apply(make_bar_text, axis=1)

# Balkendiagramm: Potenzial vs. Verbrauch vs. Produktion
fig_vergleich = px.bar(
    vergleich_df,
    x="Gruppe",
    y="GWh",
    color="Kategorie",
    text="Text",
    color_discrete_map={
        "Dächer": "#000000",
        "Fassaden": "#A7A7A7",
        "Strom-Endverbrauch": "#636efa"
    },
    title="Vergleich: Potenzielle Solarenergie (2025) vs. Strom-Endverbrauch (2024) vs. aktuelle Stromproduktion (2024)",
)

fig_vergleich.update_traces(
    textposition='inside',
    textfont_size=16,
    insidetextanchor='middle',
    showlegend=False
)
fig_vergleich.update_layout(
    barmode="stack",
    showlegend=False,
    yaxis_title="GWh",
    xaxis_title="",
    font=dict(family="Arial", size=16),
    plot_bgcolor="white",
    xaxis=dict(showline=True, linewidth=1, linecolor='black', showgrid=False, zeroline=False),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', showgrid=True, zeroline=False, gridcolor="#E0E0E0"),
    margin=dict(l=40, r=20, t=60, b=40)
)

# Diagramm anzeigen und als HTML speichern
fig_vergleich.show()
fig_vergleich.write_html("../docs/assets/diagramme/pot_solarenergie_vs_strom_endverbrauch.html")